# Using GraphRAG-SDK to Create a Knowledge Graph and RAG System from Unstructured Documents

GraphRAG-SDK provides a powerful tool, enhanced by LLM technology, to build a Retrieval-Augmented Generation (RAG) system. This example demonstrates how to load UFC HTML files, automatically detect ontology based on 10% of the files, and create a Knowledge Graph (KG) to enable a question-answerable RAG system.

In [1]:
!pip install graphrag_sdk[openai]

In [ ]:
import os
import json
import random
from dotenv import load_dotenv
from graphrag_sdk.source import Source
from graphrag_sdk import KnowledgeGraph, Ontology
from graphrag_sdk.models.litellm import LiteModel
from graphrag_sdk.model_config import KnowledgeGraphModelConfig

# Load environment variables
load_dotenv()

# Configuration
# Set your OpenAI credential in .env file
# OPENAI_API_KEY = "your-api-key"

True

### Import Source Data from Disk

This example uses UFC HTML files as the source data. We will import these files as `Source` objects.

In [ ]:
# Data folder.
src_files = "examples/ufc/data/fight"
sources = []

# For each file in the source directory, create a new Source object.
for file in os.listdir(src_files):
    sources.append(Source(os.path.join(src_files, file)))

### Ontology from the Sources

Next, we will utilize an LLM to automatically extract ontology from a portion of the data (10%) and save it as a JSON file for manual review. We will also add `boundaries` to the ontology detection process to ensure the desired ontology is accurately identified.

In [4]:
# Define the percentage of files that will be used to auto-create the ontology.
percent = 0.1  # This represents 10%. You can adjust this value (e.g., 0.2 for 20%).

boundaries = """
    Extract only the most relevant information about UFC fighters, fights, and events.
    Avoid creating entities for details that can be expressed as attributes.
"""

# Define the model to be used for the ontology
# Using OpenAI with gpt-4.1 as default (temperature=0 by default)
model = LiteModel()

# Randomly select a percentage of files from sources.
sampled_sources = random.sample(sources, round(len(sources) * percent))

ontology = Ontology.from_sources(
    sources=sampled_sources,
    boundaries=boundaries,
    model=model,
)

# Save the ontology to the disk as a json file.
with open("ontology.json", "w", encoding="utf-8") as file:
    file.write(json.dumps(ontology.to_json(), indent=2))

Process Documents: 100%|██████████| 2/2 [00:35<00:00, 17.79s/it]


Review the Ontology

In [5]:
print(json.dumps(ontology.to_json(), indent=4))

{
    "entities": [
        {
            "label": "Event",
            "attributes": [
                {
                    "name": "name",
                    "type": "string",
                    "unique": true,
                    "required": true
                },
                {
                    "name": "date",
                    "type": "string",
                    "unique": false,
                    "required": true
                },
                {
                    "name": "location",
                    "type": "string",
                    "unique": false,
                    "required": true
                }
            ],
            "description": ""
        },
        {
            "label": "Fight",
            "attributes": [
                {
                    "name": "fight_id",
                    "type": "string",
                    "unique": true,
                    "required": true
                },
                {
                    "name

### KG from Sources and Ontology

After reviewing the ontology, we will load it and use it to create a Knowledge Graph (KG) from the sources.

In [6]:
# After approving the ontology, load it from disk.
ontology_file = "ontology.json"
with open(ontology_file, "r", encoding="utf-8") as file:
    ontology = Ontology.from_json(json.loads(file.read()))

kg = KnowledgeGraph(
    name="ufc",
    model_config=KnowledgeGraphModelConfig.with_model(model),
    ontology=ontology,
)
kg.process_sources(sources)

Process Documents: 100%|██████████| 8/8 [01:18<00:00,  9.77s/it]


### Graph RAG

At this point, we have a Knowledge Graph based on our data, and we can use it in our GraphRAG system. Utilize the method `chat_session` method for starting a conversation.

In [7]:
# Conversation.
chat = kg.chat_session()
response = chat.send_message("Who is Salsa Boy?")
print(response)
response = chat.send_message("Tell me about one of his fights?")
print(response)

{'question': 'Who is Salsa Boy?', 'response': 'Salsa Boy is Waldo Cortes-Acosta.', 'context': '[[\'(:Person{name:"Waldo Cortes-Acosta",nickname:"Salsa Boy"})\']]', 'cypher': "\nMATCH (p:Person)\nWHERE p.nickname CONTAINS 'Salsa Boy'\nRETURN p\n"}
{'question': 'Tell me about one of his fights?', 'response': 'Waldo Cortes-Acosta, also known as Salsa Boy, fought Lukasz Brzeski at UFC Fight Night: Holloway vs. The Korean Zombie on August 26, 2023. He won the fight by KO/TKO with punches to the head at distance in the first round at 3:01. The bout was in the Heavyweight division and was scheduled for 3 rounds.', 'context': '[[\'(:Person{name:"Waldo Cortes-Acosta",nickname:"Salsa Boy"})\', \'()-[:PARTICIPATED{result:"Win"}]->()\', \'(:Fight{details:"Punches to Head At Distance",fight_id:"Waldo Cortes-Acosta v Lukasz Brzeski UFC Fight Night: Holloway vs. The Korean Zombie 2023-08-26",method:"KO/TKO",round:1,time:"3:01",time_format:"3 Rounds (5-5-5)",title_bout:False,weight_class:"Heavyweight"